**Plot shot gather of modelling results**<br />
Daniel Köhn<br />
Kiel, 09.07.2017

**Import necessary packages**

In [ ]:
import obspy
from obspy import read
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib.pyplot import gca
import matplotlib as mpl
from pylab import rcParams
from matplotlib import rc
#%matplotlib notebook

**Define parameters for data visualization**

In [ ]:
clip = 2e-7     # data clip
shotno = 1      # shot number

**Font properties**

In [ ]:
FSize = 20
font = {'color':  'black',
        'weight': 'bold',
        'size': FSize}
mpl.rc('xtick', labelsize=FSize) 
mpl.rc('ytick', labelsize=FSize) 
rcParams['figure.figsize'] = 10, 10

**Read modelled Marmousi-2 data**

In [ ]:
str_shotno = "%0.*f" %(0,np.fix(shotno))    # shotnumber2str
filename = "../su/DENISE_MARMOUSI_y.su.shot" + str_shotno
data = read(filename, unpack_trace_headers=True)

**Apply lowpass Butterworth filter**

In [ ]:
#data.filter('lowpass', freq=2.0, corners=6, zerophase=False)

**Extract traces and header information from data stream**

In [ ]:
# number of traces, time samples and sample interval
ntr = len(data.traces)
ns = len(data.traces[0].data)                                   
dt = data.traces[0].stats.delta                                 

# x- and y-source coordinates from trace header
xsrc = data.traces[0].stats.su.trace_header.source_coordinate_x
ysrc = data.traces[0].stats.su.trace_header.source_coordinate_y

# allocate memory for traces and receiver positions
traces = np.zeros((ns, ntr))
xrec = np.zeros((1, ntr))
yrec = np.zeros((1, ntr))

i=0
for tr in data.traces:
    
    # extract traces
    traces[:, i] = tr.data[:]
    
    # x- and y-receiver coordinates from trace header
    xrec[0,i] = data.traces[i].stats.su.trace_header.group_coordinate_x
    yrec[0,i] = data.traces[i].stats.su.trace_header.group_coordinate_y
    
    i += 1

# flip traces
traces = np.flipud(traces)
    
# offset [km]    
offset = (xrec - xsrc) / 1e6

**Plot shot gather**

In [ ]:
extent = [np.min(offset), np.max(offset), dt, dt*ns]

fig = plt.figure

plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rc('text', usetex=True)

im = plt.imshow(traces, cmap=plt.cm.seismic, interpolation='bicubic',
                extent=extent, vmin=-clip, vmax=clip)

a = gca()
a.set_xticklabels(a.get_xticks(), font)
a.set_yticklabels(a.get_yticks(), font)
#plt.axis('scaled')
title_name = "Marmousi-2 (shot no." + str_shotno + ")"
plt.title(title_name, fontdict=font)
plt.ylabel('Time [s]', fontdict=font)
plt.xlabel('Offset [km]', fontdict=font)
plt.gca().invert_yaxis()

plt.tight_layout()
output_file = "Marmousi_shot_" + str_shotno + ".pdf"
plt.savefig(output_file, bbox_inches='tight', format='pdf')
plt.show()